In [1]:
import readfof
import readgadget
import numpy as np
import redshift_space_library as RSL
import MAS_library as MASL
import os 
from tqdm import tqdm
import matplotlib.pyplot as plt
import BFast
import numpy as np
import jax
import jax.numpy as jnp
import Pk_library as PKL


Load the model and weights

In [2]:
#load the model and weights
import pickle
from UNET_jax_eff import UNET3D_jax_e
from collections import defaultdict
import optax
import joblib

BoxSize = 1000.
model = UNET3D_jax_e(image_size=256, BoxSize=1000, n_base_filters=9, depth=6, dropout_rate=0.3)


CHECKPOINT_PATH = "train_250_val_130"
# params_file = os.path.join(CHECKPOINT_PATH, "UNET3D_jax_e/model.pkl") 
params_file = os.path.join(CHECKPOINT_PATH, "UNET3D_jax_e/model.joblib")# Assuming the parameters are saved as "model_0.pkl"


with open(params_file, 'rb') as f:
    loaded_data = joblib.load(f)
    params = loaded_data['params']
    batch_stats = loaded_data['batch_stats']

def predict(model, params, batch_stats, X):
    return model.apply({'params': params, 'batch_stats': batch_stats}, X, mutable=False, training =False)
    
 


In [ ]:
import jax.numpy as jnp
from jax.numpy.fft import rfftn, irfftn
grid = 256
def cutfield(delta, BoxSize, grid, maxk_kF):
    # Cuts the density field at kmax = maxk_kF * kF 
    kF = 2 * jnp.pi / BoxSize
    cell_size = BoxSize / grid

    # Create the k-space grid
    kx = 2 * jnp.pi * jnp.fft.fftfreq(grid, cell_size)
    ky = 2 * jnp.pi * jnp.fft.fftfreq(grid, cell_size)
    kz = 2 * jnp.pi * jnp.fft.rfftfreq(grid, cell_size)
    kx, ky, kz = jnp.meshgrid(kx, ky, kz, indexing="ij")
    kgrid = jnp.sqrt(kx**2 + ky**2 + kz**2)

    # Create a boolean mask for the cut-off
    bools = (kgrid >= maxk_kF * kF)

    # Apply Fourier Transform, filter and then apply Inverse Fourier Transform
    c_fftgrid = rfftn(delta)
    c_fftgrid = jnp.where(bools, 0.+0.j, c_fftgrid)
    r_fftgrid = irfftn(c_fftgrid)

    return r_fftgrid


from jax import vmap
import jax.numpy as jnp
from jax.numpy.fft import rfftn, irfftn

# Vectorize cutfield for batch processing
batch_cutfield = vmap(cutfield, in_axes=(0, None, None, None))

# Assuming y_data is a JAX array and BoxSize, grid are defined
batch_size = 10

y_data =  np.array([np.load(f"/scratch/s3487202/Matter_Data/fiducial/0/df_m_256_PCS_fiducial_127_{i}.npy") for i in tqdm(range(100))])
print('data had loaded')
# Process each batch and normalize it before moving to the next batch
y_data_cut = np.zeros((y_data.shape[0],y_data.shape[1],y_data.shape[2],y_data.shape[3]),dtype=np.float32)
for i in range(0, y_data.shape[0], batch_size):
    print(i)
    batch = y_data[i:i + batch_size]
    processed_batch = batch_cutfield(batch, BoxSize, grid, 82.5)

    # Normalize the processed batch
    processed_batch /= processed_batch.reshape(processed_batch.shape[0], -1).std()
    processed_batch = np.array(processed_batch)

    y_data_cut[i:i + batch_size] = processed_batch



In [ ]:
# %%time
# #single example

# import logging
# logging.getLogger('jax').setLevel(logging.ERROR)

# # Filter out specific warning messages
# # warnings.filterwarnings("ignore", message="Trying algorithm.*is taking a while...")
# # warnings.filterwarnings("ignore", message="The operation took.*")
# BoxSize = 1000.
# grid = int(256)
# Hubble = int(100)
# redshift = 0
# snapshot_number = 0
# typey= 'fiducial'
# snapdir = f'/scratch/hb-CosmoGroup/Quijote_Halos/{typey}/5' #folder hosting the catalogue
# snapnum = 4   

# delta_z = np.array(y_data[0])
# axis = 2
# Pks_32_z_s = BFast.Pk(delta_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
# Bks_32_z_s = BFast.Bk(delta_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
# numbers = list(range(100)) 

# Pk_z0_array_z = np.zeros((len(numbers), Pks_32_z_s.shape[0], Pks_32_z_s.shape[1]))
# Bk_z0_array_z = np.zeros((len(numbers), Bks_32_z_s.shape[0], Bks_32_z_s.shape[1]))
# for num in numbers:
#     print(num)
#     delta_z = np.array(y_data_cut[num])

# #rotate the fields
# # delta_x_rot = np.transpose(delta_x, (1, 2, 0))
# # delta_y_rot = np.transpose(delta_y, (2, 0, 1))

# #predict and calculate power and bispectrum of pre and post?

# # pred_x = np.squeeze(predict(model,params,batch_stats,delta_x_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
# # pred_y = np.squeeze(predict(model,params,batch_stats,delta_y_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
# # pred_z = np.squeeze(predict(model,params,batch_stats,delta_z[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
# # print(delta_z.shape)

# #do not save the fields because of memory
# # calculate power and bispectra for fields
# # Pks_32_x_s = BFast.Pk(delta_x_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
# # Pks_32_y_s = BFast.Pk(delta_y_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
#     Pks_32_z_s = BFast.Pk(delta_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')

# # Pks_32_pred_x_s = BFast.Pk(pred_x,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
# # Pks_32_pred_y_s = BFast.Pk(pred_y,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
# # Pks_32_pred_z_s = BFast.Pk(pred_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')

# # Bks_32_x_s = BFast.Bk(delta_x_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
# # Bks_32_y_s = BFast.Bk(delta_y_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
#     Bks_32_z_s = BFast.Bk(delta_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)

# # Bks_32_pred_x_s = BFast.Bk(pred_x,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
# # Bks_32_pred_y_s = BFast.Bk(pred_y,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
# # Bks_32_pred_z_s = BFast.Bk(pred_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
#     Pk_z0_array_z[num] = Pks_32_z_s
#     Bk_z0_array_z[num] = Bks_32_z_s


# np.save(f"/scratch/s3487202/Results/power_spectrum/y_data_{typey}.npy", Pk_z0_array_z)
# np.save(f"/scratch/s3487202/Results/bispectrum/y_data_{typey}.npy", Bk_z0_array_z)
# print(Pk_z0_array_z.shape)


single example to get the eventual shape of the output

In [ ]:
%%time
#single example

import logging
logging.getLogger('jax').setLevel(logging.ERROR)

# Filter out specific warning messages
# warnings.filterwarnings("ignore", message="Trying algorithm.*is taking a while...")
# warnings.filterwarnings("ignore", message="The operation took.*")
BoxSize = 1000.
grid = int(256)
Hubble = int(100)
redshift = 0
snapshot_number = 0
typey= 'fiducial'
snapdir = f'/scratch/hb-CosmoGroup/Quijote_Halos/{typey}/5' #folder hosting the catalogue
snapnum = 4   
# read the halo catalogue

FoF = readfof.FoF_catalog(snapdir, snapnum, long_ids=False, swap=False, SFR=False, read_IDs=False)
pos_h = FoF.GroupPos / 1e3
vel_h = FoF.GroupVel*(1.0+redshift) #Halo peculiar velocities in km/s

#read the void catalogue

# Move particles to redshift-space
axis_list = [0, 1, 2]
# axis_list = [2] # z axis because void data is distorted in z axis
delta_fields = []
mean_fields = []

for axis in axis_list:
    RSL.pos_redshift_space(pos_h, vel_h, BoxSize, Hubble, redshift, axis)
    delta = np.zeros((grid, grid, grid), dtype=np.float32)
    #MASL.MA(pos_h_d, delta, BoxSize, MAS, verbose=verbose)
    MASL.MA(pos_h, delta, BoxSize, 'PCS', verbose=False)
    # mean_fields.append(np.mean(delta, dtype=np.float32))
    delta /= np.mean(delta, dtype=np.float32)
    delta -= 1.0
    delta_fields.append(delta)

# Separate the density fields for x, y, and z axes
delta_x,delta_y,delta_z = delta_fields

delta_x = np.array(delta_x)
delta_y = np.array(delta_y)
delta_z = np.array(delta_z)

#rotate the fields
delta_x_rot = np.transpose(delta_x, (1, 2, 0))
delta_y_rot = np.transpose(delta_y, (2, 0, 1))

#predict and calculate power and bispectrum of pre and post?

pred_x = np.squeeze(predict(model,params,batch_stats,delta_x_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
pred_y = np.squeeze(predict(model,params,batch_stats,delta_y_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
pred_z = np.squeeze(predict(model,params,batch_stats,delta_z[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
print(delta_z.shape)

#do not save the fields because of memory
# calculate power and bispectra for fields
Pks_32_x_s = BFast.Pk(delta_x_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
Pks_32_y_s = BFast.Pk(delta_y_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
Pks_32_z_s = BFast.Pk(delta_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')

Pks_32_pred_x_s = BFast.Pk(pred_x,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
Pks_32_pred_y_s = BFast.Pk(pred_y,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
Pks_32_pred_z_s = BFast.Pk(pred_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')

Bks_32_x_s = BFast.Bk(delta_x_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
Bks_32_y_s = BFast.Bk(delta_y_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
Bks_32_z_s = BFast.Bk(delta_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)

Bks_32_pred_x_s = BFast.Bk(pred_x,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
Bks_32_pred_y_s = BFast.Bk(pred_y,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
Bks_32_pred_z_s = BFast.Bk(pred_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)

print(Pks_32_pred_x_s.shape)
print(Bks_32_pred_x_s.shape)


Code to calculate the spectra of the derivatives( non fiducial cosmology simulations in my case)

In [ ]:
%%time
#for derivatives
BoxSize = 1000.
grid = int(256)
Hubble = int(100)
redshift = 0
# numbers = list(range(81)) 

snapnum = 4
kF = 2*np.pi/BoxSize
threads=8
axis =2

numbers = list(range(500)) 
Pk_z0_array_x = np.zeros((len(numbers),  Pks_32_x_s.shape[0],Pks_32_x_s.shape[1],))
Pk_z127_rec_array_x = np.zeros((len(numbers), Pks_32_pred_x_s.shape[0],Pks_32_pred_x_s.shape[1],))

Pk_z0_array_y = np.zeros((len(numbers), Pks_32_y_s.shape[0], Pks_32_y_s.shape[1]))
Pk_z127_rec_array_y = np.zeros((len(numbers), Pks_32_pred_y_s.shape[0], Pks_32_pred_y_s.shape[1]))

Pk_z0_array_z = np.zeros((len(numbers), Pks_32_z_s.shape[0], Pks_32_z_s.shape[1]))
Pk_z127_rec_array_z = np.zeros((len(numbers), Pks_32_pred_z_s.shape[0], Pks_32_pred_z_s.shape[1]))

Bk_z0_array_x = np.zeros((len(numbers),  Bks_32_x_s.shape[0],Bks_32_x_s.shape[1],))
Bk_z127_rec_array_x = np.zeros((len(numbers), Bks_32_pred_x_s.shape[0],Bks_32_pred_x_s.shape[1],))

Bk_z0_array_y = np.zeros((len(numbers), Bks_32_y_s.shape[0], Bks_32_y_s.shape[1]))
Bk_z127_rec_array_y = np.zeros((len(numbers), Bks_32_pred_y_s.shape[0], Bks_32_pred_y_s.shape[1]))

Bk_z0_array_z = np.zeros((len(numbers), Bks_32_z_s.shape[0], Bks_32_z_s.shape[1]))
Bk_z127_rec_array_z = np.zeros((len(numbers), Bks_32_pred_z_s.shape[0], Bks_32_pred_z_s.shape[1]))
# typey_list = ['Ob2_p','Om_m','Om_p','OR_LSS_m','OR_LSS_p','s8_m','s8_p']
typey_list = ['EQ_m','EQ_p','h_m','h_p','LC_m','LC_p']
# typey = 'EQ_m'
for typey in typey_list:
    print(f'loop {typey} starts ')
    for num in numbers:
        print(num)
        snapshot_number = num
        snapdir = f'/scratch/hb-CosmoGroup/Quijote_Halos/{typey}/{snapshot_number}' #folder hosting the catalogue
       
        # read the halo catalogue
    
        FoF = readfof.FoF_catalog(snapdir, snapnum, long_ids=False, swap=False, SFR=False, read_IDs=False)
        pos_h = FoF.GroupPos / 1e3
        vel_h = FoF.GroupVel*(1.0+redshift) #Halo peculiar velocities in km/s
    
        #read the void catalogue
    
        # Move particles to redshift-space
        axis_list = [0, 1, 2]
        # axis_list = [2] # z axis because void data is distorted in z axis
        delta_fields = []
        mean_fields = []
        for axis in axis_list:

            # Create a copy of pos_h for each axis
            pos_h_axis = pos_h.copy()
            RSL.pos_redshift_space(pos_h_axis, vel_h, BoxSize, Hubble, redshift, axis)
            delta = np.zeros((grid, grid, grid), dtype=np.float32)
            MASL.MA(pos_h_axis, delta, BoxSize, 'PCS', verbose=False)
            delta /= np.mean(delta, dtype=np.float32)
            delta -= 1.0
            delta_fields.append(delta)
        
        
        # Separate the density fields for x, y, and z axes
        delta_x,delta_y,delta_z = delta_fields
        # print(delta_z.shape)
        delta_x = np.array(delta_x)
        delta_y = np.array(delta_y)
        delta_z = np.array(delta_z)
    
        #rotate the fields
        delta_x_rot = np.transpose(delta_x, (1, 2, 0))
        delta_y_rot = np.transpose(delta_y, (2, 0, 1))
    
        
        #predict and calculate power and bispectrum of pre and post?
        axis = 2
        pred_x = np.squeeze(predict(model,params,batch_stats,delta_x_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
        pred_y = np.squeeze(predict(model,params,batch_stats,delta_y_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
        pred_z = np.squeeze(predict(model,params,batch_stats,delta_z[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
            
        #do not save the fields because of memory
        # calculate power and bispectra for fields
        Pks_32_x = BFast.Pk(delta_x_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_y = BFast.Pk(delta_y_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_z = BFast.Pk(delta_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
    
        Pks_32_pred_x = BFast.Pk(pred_x,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_pred_y = BFast.Pk(pred_y,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_pred_z = BFast.Pk(pred_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
    
        Bks_32_x = BFast.Bk(delta_x_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_y = BFast.Bk(delta_y_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_z = BFast.Bk(delta_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
    
        Bks_32_pred_x = BFast.Bk(pred_x,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_pred_y = BFast.Bk(pred_y,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_pred_z = BFast.Bk(pred_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
    
    
        Pk_z0_array_x[num]= Pks_32_x
        Pk_z127_rec_array_x[num] = Pks_32_pred_x
    
        Pk_z0_array_y[num] = Pks_32_y
        Pk_z127_rec_array_y[num] = Pks_32_pred_y
    
        Pk_z0_array_z[num] = Pks_32_z
        Pk_z127_rec_array_z[num] = Pks_32_pred_z
    
        Bk_z0_array_x[num] = Bks_32_x
        Bk_z127_rec_array_x[num] = Bks_32_pred_x
    
        Bk_z0_array_y[num] = Bks_32_y
        Bk_z127_rec_array_y[num] = Bks_32_pred_y
    
        Bk_z0_array_z[num] = Bks_32_z
        Bk_z127_rec_array_z[num] = Bks_32_pred_z
    print(f'loop {typey} done')
    
    # Concatenate _array_x with _array_y and _array_z
    Pk_z0_array_combined = np.concatenate([Pk_z0_array_x, Pk_z0_array_y, Pk_z0_array_z], axis=0)
    Pk_z127_rec_array_combined = np.concatenate([Pk_z127_rec_array_x, Pk_z127_rec_array_y, Pk_z127_rec_array_z], axis=0)
    
    Bk_z0_array_combined = np.concatenate([Bk_z0_array_x, Bk_z0_array_y, Bk_z0_array_z], axis=0)
    Bk_z127_rec_array_combined = np.concatenate([Bk_z127_rec_array_x, Bk_z127_rec_array_y, Bk_z127_rec_array_z], axis=0)
    
    # for derivatives
    np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}.npy", Pk_z0_array_combined)
    np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}.npy", Pk_z127_rec_array_combined)
    np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}.npy", Bk_z0_array_combined)
    np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}.npy", Bk_z127_rec_array_combined)

    #for fiducial
    # np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}_0-2499.npy", Pk_z0_array_z)
    # np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}_0-2499.npy", Pk_z127_rec_array_z)
    # np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}_0-2499.npy", Bk_z0_array_z)
    # np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}_0-2499.npy", Bk_z127_rec_array_z)
    print(f'{typey} saved')
print(Bk_z0_array_combined.shape)    

Code for calculating the spectra of the fiducial simulations in batches, used for the covariance

In [ ]:
%%time

#for fiducial
BoxSize = 1000.
grid = int(256)
Hubble = int(100)
redshift = 0


snapnum = 4
kF = 2*np.pi/BoxSize
threads=8
axis =2

numbers = list(range(10000,15000)) 
# Pk_z0_array_x = np.zeros((len(numbers),  Pks_32_x_s.shape[0],Pks_32_x_s.shape[1],))
# Pk_z127_rec_array_x = np.zeros((len(numbers), Pks_32_pred_x_s.shape[0],Pks_32_pred_x_s.shape[1],))

# Pk_z0_array_y = np.zeros((len(numbers), Pks_32_y_s.shape[0], Pks_32_y_s.shape[1]))
# Pk_z127_rec_array_y = np.zeros((len(numbers), Pks_32_pred_y_s.shape[0], Pks_32_pred_y_s.shape[1]))

Pk_z0_array_z = np.zeros((len(numbers), Pks_32_z_s.shape[0], Pks_32_z_s.shape[1]))
Pk_z127_rec_array_z = np.zeros((len(numbers), Pks_32_pred_z_s.shape[0], Pks_32_pred_z_s.shape[1]))

# Bk_z0_array_x = np.zeros((len(numbers),  Bks_32_x_s.shape[0],Bks_32_x_s.shape[1],))
# Bk_z127_rec_array_x = np.zeros((len(numbers), Bks_32_pred_x_s.shape[0],Bks_32_pred_x_s.shape[1],))

# Bk_z0_array_y = np.zeros((len(numbers), Bks_32_y_s.shape[0], Bks_32_y_s.shape[1]))
# Bk_z127_rec_array_y = np.zeros((len(numbers), Bks_32_pred_y_s.shape[0], Bks_32_pred_y_s.shape[1]))

Bk_z0_array_z = np.zeros((len(numbers), Bks_32_z_s.shape[0], Bks_32_z_s.shape[1]))
Bk_z127_rec_array_z = np.zeros((len(numbers), Bks_32_pred_z_s.shape[0], Bks_32_pred_z_s.shape[1]))
# typey_list = ['Ob2_p','Om_m','Om_p','OR_LSS_m','OR_LSS_p','s8_m','s8_p']
typey_list = ['fiducial']
# typey = 'EQ_p'
for typey in typey_list:
    print(f'loop {typey} starts ')
    for num in numbers:
        print(num)
        snapshot_number = num
        snapdir = f'/scratch/hb-CosmoGroup/Quijote_Halos/{typey}/{snapshot_number}' #folder hosting the catalogue
       
        # read the halo catalogue
    
        FoF = readfof.FoF_catalog(snapdir, snapnum, long_ids=False, swap=False, SFR=False, read_IDs=False)
        pos_h = FoF.GroupPos / 1e3
        vel_h = FoF.GroupVel*(1.0+redshift) #Halo peculiar velocities in km/s
    
        #read the void catalogue
    
        # Move particles to redshift-space
        # axis_list = [0, 1, 2]
        axis_list = [2] # z axis because void data is distorted in z axis
        delta_fields = []
        mean_fields = []
        # for axis in axis_list:
        RSL.pos_redshift_space(pos_h, vel_h, BoxSize, Hubble, redshift, axis)
        delta = np.zeros((grid, grid, grid), dtype=np.float32)
        #MASL.MA(pos_h_d, delta, BoxSize, MAS, verbose=verbose)
        MASL.MA(pos_h, delta, BoxSize, 'PCS', verbose=False)
        # mean_fields.append(np.mean(delta, dtype=np.float32))
        delta /= np.mean(delta, dtype=np.float32)
        delta -= 1.0
        delta_z = delta
        # delta_fields.append(delta)
        
        
        # Separate the density fields for x, y, and z axes
        #delta_x,delta_y,delta_z = delta_fields
        # delta_z = delta_fields
    
        # delta_x = np.array(delta_x)
        # delta_y = np.array(delta_y)
        delta_z = np.array(delta_z)
 
        #rotate the fields
        # delta_x_rot = np.transpose(delta_x, (1, 2, 0))
        # delta_y_rot = np.transpose(delta_y, (2, 0, 1))
    
        
        #predict and calculate power and bispectrum of pre and post?
        axis = 2
        # pred_x = np.squeeze(predict(model,params,batch_stats,delta_x_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
        # pred_y = np.squeeze(predict(model,params,batch_stats,delta_y_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
        pred_z = np.squeeze(predict(model,params,batch_stats,delta_z[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
            
        #do not save the fields because of memory
        # calculate power and bispectra for fields
        # Pks_32_x = BFast.Pk(delta_x_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        # Pks_32_y = BFast.Pk(delta_y_rot,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_z = BFast.Pk(delta_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
    
        # Pks_32_pred_x = BFast.Pk(pred_x,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        # Pks_32_pred_y = BFast.Pk(pred_y,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
        Pks_32_pred_z = BFast.Pk(pred_z,1000.,axis,MAS='PCS',left_inclusive=True,precision='float32')
    
        # Bks_32_x = BFast.Bk(delta_x_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        # Bks_32_y = BFast.Bk(delta_y_rot,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_z = BFast.Bk(delta_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
    
        # Bks_32_pred_x = BFast.Bk(pred_x,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        # Bks_32_pred_y = BFast.Bk(pred_y,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
        Bks_32_pred_z = BFast.Bk(pred_z,BoxSize,3.,3.,27,'All',MAS='PCS',fast=True,precision='float32',verbose=False)
    
    
        # Pk_z0_array_x[num]= Pks_32_x
        # Pk_z127_rec_array_x[num] = Pks_32_pred_x
    
        # Pk_z0_array_y[num] = Pks_32_y
        # Pk_z127_rec_array_y[num] = Pks_32_pred_y
    
        Pk_z0_array_z[num-10000] = Pks_32_z
        Pk_z127_rec_array_z[num-10000] = Pks_32_pred_z
    
        # Bk_z0_array_x[num] = Bks_32_x
        # Bk_z127_rec_array_x[num] = Bks_32_pred_x
    
        # Bk_z0_array_y[num] = Bks_32_y
        # Bk_z127_rec_array_y[num] = Bks_32_pred_y
    
        Bk_z0_array_z[num-10000] = Bks_32_z
        Bk_z127_rec_array_z[num-10000] = Bks_32_pred_z
    print(f'loop {typey} done')
    
    # Concatenate _array_x with _array_y and _array_z
    # Pk_z0_array_combined = np.concatenate([Pk_z0_array_x, Pk_z0_array_y, Pk_z0_array_z], axis=0)
    # Pk_z127_rec_array_combined = np.concatenate([Pk_z127_rec_array_x, Pk_z127_rec_array_y, Pk_z127_rec_array_z], axis=0)
    
    # Bk_z0_array_combined = np.concatenate([Bk_z0_array_x, Bk_z0_array_y, Bk_z0_array_z], axis=0)
    # Bk_z127_rec_array_combined = np.concatenate([Bk_z127_rec_array_x, Bk_z127_rec_array_y, Bk_z127_rec_array_z], axis=0)
    
    #for derivatives
    # np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}.npy", Pk_z0_array_combined)
    # np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}.npy", Pk_z127_rec_array_combined)
    # np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}.npy", Bk_z0_array_combined)
    # np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}.npy", Bk_z127_rec_array_combined)

    #for fiducial
    np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}_10000-14999.npy", Pk_z0_array_z)
    np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}_10000-14999.npy", Pk_z127_rec_array_z)
    np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}_10000-14999.npy", Bk_z0_array_z)
    np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}_10000-14999.npy", Bk_z127_rec_array_z)
    print(f'{typey} saved')

In [12]:
np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}.npy", Pk_z0_array_combined)
np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}.npy", Pk_z127_rec_array_combined)
np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}.npy", Bk_z0_array_combined)
np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}.npy", Bk_z127_rec_array_combined)

In [ ]:
print(Pk_z0_array_combined[0])
print(Pk_z127_rec_array_combined[0])
print(Pk_z127_rec_array_combined[500])

In [ ]:
# %%time

# def initialize_arrays(shape, dtype=np.float32):
#     return np.zeros((len(numbers), *shape), dtype=dtype)

# def process_catalog(snapshot_dir, snapshot_number, BoxSize, grid, redshift, Hubble):
#     FoF = readfof.FoF_catalog(snapshot_dir, snapshot_number, long_ids=False, swap=False, SFR=False, read_IDs=False)
#     pos_h = FoF.GroupPos / 1e3
#     vel_h = FoF.GroupVel * (1.0 + redshift)
#     return pos_h, vel_h

# def calculate_spectra(delta_fields, BoxSize, axis):
#     pks, bks = [], []
#     rotation_map = {
#         0: (1, 2, 0),  # x-axis
#         1: (2, 0, 1),  # y-axis
#         2: (0, 1, 2)   # z-axis
#     }
#     for i, field in enumerate(delta_fields):
 
#         field_rot = np.transpose(field, rotation_map[i])
#         pks.append(BFast.Pk(field_rot, BoxSize, 2, MAS='PCS', left_inclusive=True, precision='float32'))
#         bks.append(BFast.Bk(field_rot, BoxSize, 3., 3., 27, 'All', MAS='PCS', fast=True, precision='float32', verbose=False))
#     return pks, bks

# def predict_and_calculate_spectra(model, params, batch_stats, delta_fields):
#     spectra = []
#     for i, delta in enumerate(delta_fields):
 
#         rotation_map = {
#             0: (1, 2, 0),  # x-axis
#             1: (2, 0, 1),  # y-axis
#             2: (0, 1, 2)   # z-axis
#         }
#         delta_rot = np.transpose(delta, rotation_map[i])
#         pred = np.squeeze(predict(model, params, batch_stats, delta_rot[np.newaxis,:,:,:,np.newaxis])[0,:,:,:])
#         pks = BFast.Pk(pred, 1000., 2, MAS='PCS', left_inclusive=True, precision='float32')
#         bks = BFast.Bk(pred, 1000., 3., 3., 27, 'All', MAS='PCS', fast=True, precision='float32', verbose=False)
#         spectra.append((pks, bks))
#     return spectra

# def apply_redshift_space_distortion(pos_h, vel_h, BoxSize, Hubble, redshift, grid, axis_list):
#     delta_fields = []
#     for axis in axis_list:
#         delta = np.zeros((grid, grid, grid), dtype=np.float32)
#         RSL.pos_redshift_space(pos_h, vel_h, BoxSize, Hubble, redshift, axis)
#         MASL.MA(pos_h, delta, BoxSize, 'PCS', verbose=False)
#         delta /= np.mean(delta)
#         delta -= 1.0
#         delta_fields.append(delta)
#     return delta_fields

# # Setup and initialization as before
# BoxSize, grid, Hubble, redshift = 1000.0, 256, 100, 0
# numbers = list(range(10))
# axis_list = [0, 1, 2]
# typey, snapnum = 'EQ_m', 4

# # Initialize arrays
# Pk_z0_arrays = [initialize_arrays(Pks_32_x_s.shape) for _ in range(3)]
# Pk_z127_rec_arrays = [initialize_arrays(Pks_32_pred_x_s.shape) for _ in range(3)]
# Bk_z0_arrays = [initialize_arrays(Bks_32_x_s.shape) for _ in range(3)]
# Bk_z127_rec_arrays = [initialize_arrays(Bks_32_pred_x_s.shape) for _ in range(3)]


# # Processing loop
# print('loop starts')
# for num in numbers:
#     print(num)
#     snapdir = f'/scratch/hb-CosmoGroup/Quijote_Halos/{typey}/{num}'
#     pos_h, vel_h = process_catalog(snapdir, snapnum, BoxSize, grid, redshift, Hubble)
#     delta_fields = apply_redshift_space_distortion(pos_h, vel_h, BoxSize, Hubble, redshift, grid, axis_list)
#     pks, bks = calculate_spectra(delta_fields, BoxSize, axis_list)

#     pks_original, bks_original = calculate_spectra(delta_fields, BoxSize, 2)
#     spectra_predicted = predict_and_calculate_spectra(model, params, batch_stats, delta_fields)
    
#     Pk_z0_array_x[num], Pk_z127_rec_array_x[num] = pks_original[0], spectra_predicted[0][0]
#     Pk_z0_array_y[num], Pk_z127_rec_array_y[num] = pks_original[1], spectra_predicted[1][0]
#     Pk_z0_array_z[num], Pk_z127_rec_array_z[num] = pks_original[2], spectra_predicted[2][0]
    
#     Bk_z0_array_x[num], Bk_z127_rec_array_x[num] = bks_original[0], spectra_predicted[0][1]
#     Bk_z0_array_y[num], Bk_z127_rec_array_y[num] = bks_original[1], spectra_predicted[1][1]
#     Bk_z0_array_z[num], Bk_z127_rec_array_z[num] = bks_original[2], spectra_predicted[2][1]

# print('loop done')

# Pk_z0_combined = np.concatenate(Pk_z0_arrays, axis=0)
# Pk_z127_rec_combined = np.concatenate(Pk_z127_rec_arrays, axis=0)
# Bk_z0_combined = np.concatenate(Bk_z0_arrays, axis=0)
# Bk_z127_rec_combined = np.concatenate(Bk_z127_rec_arrays, axis=0)




In [ ]:
print(Pk_z0_combined.shape)
print(Bk_z127_rec_combined.shape)
print(Pk_z127_rec_combined[0])
print(Pk_z127_rec_combined[5])


In [8]:
#save the spectra data with correct numbers
# 
np.save(f"/scratch/s3487202/Results/power_spectrum/halo_data_z0_{typey}.npy", Pk_z0_combined)
np.save(f"/scratch/s3487202/Results/power_spectrum/rec_z127_{typey}.npy", Pk_z127_rec_combined)
np.save(f"/scratch/s3487202/Results/bispectrum/halo_data_z0_{typey}.npy", Bk_z0_combined)
np.save(f"/scratch/s3487202/Results/bispectrum/rec_z127_{typey}.npy", Bk_z127_rec_combined)


In [9]:
print( Pk_z0_combined.shape)
print(Bk_z127_rec_combined.shape)

(1500, 220, 5)
(1500, 2276, 8)
